In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np
from grating import *
from ext_signal import *
from scipy.stats import qmc

lum in LMS = [0.45436904 0.47353315 0.0720978 ]
[[1.31265115 0.37916654 0.17658182]
 [0.37916654 0.43588297 0.06613005]
 [0.17658182 0.06613005 1.03101131]]
[[10.82668925 -2.48855249  0.0461736 ]
 [-2.48855249  1.38614356 -0.12768065]
 [ 0.0461736  -0.12768065  1.15767798]]


In [2]:
def acuity(ecc):
    k = 0.2049795945022049
    log_cpd0 = 3.6741080244555278
    cpd = np.exp(-k*ecc + log_cpd0)
    return 2/cpd/4

acuity(0.08)

0.01289575827657954

In [3]:
K_onC = 29.13289963
K_onS = 23.17378917
K_offC = 22.71914498
K_offS = 12.53276353
r_C = (acuity(0.095) + acuity(0))/2
r_S = r_C*3
cov = 0.53753
def zeroCross(k1, k2, r1, r2):
    return np.sqrt(np.log(k1/k2)/(r2*r2 - r1*r1))*r1*r2
SF = 1.0/(zeroCross(K_offC, K_onS*cov, r_C, r_S)*2 + zeroCross(K_onC, K_offS*cov, r_C, r_S)*2)
print(SF)

18.53509145317477


In [4]:
# color-drifting
inputLMS = True
range_deg = 0.0344#0.12672379774682216 # eccentricity from the origin
print(acuity(range_deg))
buffer_deg = acuity(range_deg)*4
print(f'range_deg = {range_deg}, buffer_deg = {buffer_deg}')
SF = 20
TF = 8
time = np.array([1/TF])
neye = 1
center = np.pi/2 
wing = np.pi/2
sharpness = 1

0.012775782312929865
range_deg = 0.0344, buffer_deg = 0.05110312925171946


# 上面的参数你都不需要改，从下面开始改

In [5]:
path = '/home/zhaobenyan/data/resource_contrast32_new/' #地址,运行代码前先在终端创建文件夹
# 大写的都是列表，贮存此参数所有可选值

n = 1 # 给图片编号的序号
C = np.arange(0.05,0.55,0.05) # contrast范围在[0,0.5]
#C=np.array([0.35])

# P = np.arange(0,2*np.pi,np.pi/6)
#P = np.arange(0,1,1) #相位,范围[0,2pi),如果你只想研究contrast的话，别的参数最好固定统一
P=np.array([np.pi])

# SF = np.arange(,84,4)
SF = np.arange(20,22,2) ##spatial frequency,建议范围在10到40之间

#D = np.arange(0,np.pi,np.pi/12)
D = np.array([np.pi/6])  # 方向范围[0,pi]

C = C.astype('float32')
P = P.astype('float32')
SF = SF.astype('float32')
D = D.astype('float32')

for c in C:
    for sf in SF:
        for orient in D:
            for phase in P:
                crest = np.array([0.5+c, 0.5-c, 0.5]) #contrast和正弦光栅的波峰、波谷在LMS颜色空间的关系
                valley = np.array([0.5-c, 0.5+c, 0.5])
                video_fn = path + f'static_color-grid_{n}'
                cfg_fn = path + f'static_color-grid_{n}_cfg' #XXXcfg.bin储存每张图的参数，可以关注一下这是怎么存的，要关注存的时候的变量类型，是f4还是u4，怎么存就怎么读，
                f = open(cfg_fn+ '.bin', 'wb') # sf,d,p,c
                np.array([sf]).astype('f4').tofile(f)
                np.array([orient*180/np.pi]).astype('f4').tofile(f) #generate_grating需要弧度制，但是我参数保存的角度制
                np.array([phase*180/np.pi]).astype('f4').tofile(f)
                np.array([c]).astype('f4').tofile(f)
                crest.astype('f4').tofile(f)
                valley.astype('f4').tofile(f)
                f.close()
                generate_grating(1.0, sf, TF, orient, 32, crest, valley, video_fn, time, phase, sharpness, frameRate = 1, ecc = range_deg, buffer_ecc = buffer_deg, gtype='drifting', neye = neye, bar = False, center = center, wing = wing, mask = None, inputLMS = inputLMS,genMovie = True)
                n = n+1


32 degree per pixel
crest in sRGB: [0.91965815 0.66944248 0.70707138]
valley in sRGB: [0.64848672 0.7627348  0.70176079]
0.005343945578232466 degree per pixel
ecc = 0.0344, buffer_ecc = 0.05110312925171946
1
adjusted to 1 frames in total
sharpness=1.0
it: 0
32 degree per pixel
crest in sRGB: [1.02220551 0.61576424 0.70970734]
valley in sRGB: [0.42848606 0.80422963 0.69908582]
0.005343945578232466 degree per pixel
ecc = 0.0344, buffer_ecc = 0.05110312925171946
1
adjusted to 1 frames in total
sharpness=1.0
it: 0
sRGB space is not enough to represent the color
(array([0.42849847, 0.61576663, 0.69908594]), array([1.02220147, 0.80422794, 0.70970723]))
32 degree per pixel
crest in sRGB: [1.11263462 0.55527554 0.71233065]
valley in sRGB: [-0.91308059  0.843093    0.69639751]
0.005343945578232466 degree per pixel
ecc = 0.0344, buffer_ecc = 0.05110312925171946
1
adjusted to 1 frames in total
sharpness=1.0
it: 0
sRGB space is not enough to represent the color
(array([-0.91289707,  0.55527964,  0

# 读参数文件

In [5]:
# cfg_fn = path + f'static_color-grid_{n}_cfg' #XXXcfg.bin储存每张图的参数，可以关注一下这是怎么存的，要关注存的时候的变量类型，是f4还是u4，怎么存就怎么读，
# f = open(cfg_fn+ '.bin', 'wb') # sf,d,p,c
# np.array([sf]).astype('f4').tofile(f)
# np.array([orient*180/np.pi]).astype('f4').tofile(f)
# np.array([phase*180/np.pi]).astype('f4').tofile(f)
# np.array([c]).astype('f4').tofile(f)
# crest.astype('f4').tofile(f)
# valley.astype('f4').tofile(f)
# f.close()

#参考上面怎么写入的
path='/home/zhaobenyan/data/resource_grid_benchmark_224pixel/'
with open(path+'static_color-grid_1801_cfg.bin') as f: #可以去查查with是干嘛的，它是以防你忘记关掉文件，这种表达会比较好
    sf = np.fromfile(f,'f4',1)[0] #为啥是np.fromfile（。。。）[0],因为用np命令的读出来是array，其实我们只是要他的元素
    ori = np.fromfile(f,'f4',1)[0]
    phase = np.fromfile(f,'f4',1)[0]
    contrast = np.fromfile(f,'f4',1)[0] #我一般读取到contrast就停了
    crest = np.fromfile(f,'f4',3) 
    valley = np.fromfile(f,'f4',3)
sf,ori,phase,contrast,crest,valley

(52.0,
 90.0,
 0.0,
 0.3,
 array([0.8, 0.2, 0.5], dtype=float32),
 array([0.2, 0.8, 0.5], dtype=float32))

In [6]:
#然后思考一下怎么把参数保存在一个矩阵里
n = 10 #图片的数量
cfg = np.zeros((n,4)) #每张图片4个参数
for i in range(n):
    with open(path+f'static_color-grid_{i+1}_cfg.bin') as f: #可以去查查with是干嘛的，它是以防你忘记关掉文件，这种表达会比较好
        sf = np.fromfile(f,'f4',1)[0]
        ori = np.fromfile(f,'f4',1)[0]
        phase = np.fromfile(f,'f4',1)[0]
        contrast = np.fromfile(f,'f4',1)[0] #我一般读取到contrast就停了
    cfg[i][0] = sf
    cfg[i][1] = ori
    cfg[i][2] = phase
    cfg[i][3] = contrast
print(cfg)
    

[[  4.           0.           0.           0.30000001]
 [  4.           0.          30.           0.30000001]
 [  4.           0.          60.           0.30000001]
 [  4.           0.          90.           0.30000001]
 [  4.           0.         120.           0.30000001]
 [  4.           0.         150.           0.30000001]
 [  4.           0.         180.           0.30000001]
 [  4.           0.         210.           0.30000001]
 [  4.           0.         240.           0.30000001]
 [  4.           0.         270.           0.30000001]]


: 